<a href="https://colab.research.google.com/github/rickqiu/deep-learning/blob/master/TensorFlow2_BERT_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow 2 - BERT: Movie Review Sentiment Analysis

$BERT$ stands for Bidirectional Encoder Representations from Transformers. A pre-trained BERT model can be fine tuned to create state-of-the-art models for a wide range of NLP tasks such as question answering, sentiment analysis and named entity recognition. $BERT_{BASE}$ has total parameters of 110M (L=12, H=768, A=12, Total Parameters=110M) and $BERT_{LARGE}$ has 340M (L=24, H=1024, A=16, Total Parameters=340M) (Devlin et al, 2019, [BERT paper link](https://arxiv.org/pdf/1810.04805.pdf)).


**Dataset**

IMDB dataset has 50K movie reviews for natural language processing.
Please download the dataset from [Kaggle link](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews).

**Problem**

A review in the IMDB dataset is either positive or negative. Therefore, the NLP movie review sentiment analysis task is a supervised learning binary classification problem.

In [0]:
# Install the required package
!pip install bert-for-tf2

In [0]:
# Import modules
import pandas as pd
import numpy as np
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
pd.set_option('display.max_colwidth',1000)


TensorFlow Version: 2.2.0
Hub version:  0.8.0


## Data preprocessing

In [0]:
# Read the IMDB Dataset.csv into Pandas dataframe
df=pd.read_csv('IMDB Dataset.csv')

In [0]:
# Take a peek at the dataset
df.head(5)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the diffe

In [0]:
print("The number of rows and columns in the dataset is: {}".format(df.shape))

The number of rows and columns in the dataset is: (50000, 2)


In [0]:
# Identify missing values
df.apply(lambda x: sum(x.isnull()), axis=0)

review       0
sentiment    0
dtype: int64

In [0]:
# Check the target class balance
df["sentiment"].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [0]:
# Functions for constructing BERT Embeddings: input_ids, input_masks, input_segments and Inputs
MAX_SEQ_LEN=500 # max sequence length

def get_masks(tokens):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1]*len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))
 
def get_segments(tokens):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))

def get_ids(tokens, tokenizer):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids

def create_single_input(sentence, tokenizer, max_len):
  """Create an input from a sentence"""
  stokens = tokenizer.tokenize(sentence)
  stokens = stokens[:max_len]
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer)
  masks = get_masks(stokens)
  segments = get_segments(stokens)

  return ids, masks, segments
 
def convert_sentences_to_features(sentences, tokenizer):
  """Convert sentences to features: input_ids, input_masks and input_segments"""
  input_ids, input_masks, input_segments = [], [], []
 
  for sentence in tqdm(sentences,position=0, leave=True):
    ids,masks,segments=create_single_input(sentence,tokenizer,MAX_SEQ_LEN-2)
    assert len(ids) == MAX_SEQ_LEN
    assert len(masks) == MAX_SEQ_LEN
    assert len(segments) == MAX_SEQ_LEN
    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
          np.asarray(input_masks, dtype=np.int32), 
          np.asarray(input_segments, dtype=np.int32)]

def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case=bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer=bert.bert_tokenization.FullTokenizer(vocab_file,do_lower_case)
    return tokenizer

## Modelling

In [0]:
def nlp_model(callable_object):
    # Load saved BERT base model
    bert_layer = hub.KerasLayer(handle=callable_object, trainable=True)  
   
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")
    
    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    
    # Add a hidden layer
    x = Dense(units=768, activation='relu')(pooled_output)
    x = Dropout(0.1)(x)
 
    # Add output layer
    outputs = Dense(2, activation="softmax")(x)

    # Construct a new model
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = nlp_model("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_masks[0][0]            

## Model training

In [0]:
# Create examples for training and testing
df = df.sample(frac=1) # Shuffle the dataset
tokenizer = create_tonkenizer(model.layers[3])
X_train = convert_sentences_to_features(df['review'][:40000], tokenizer)
X_test = convert_sentences_to_features(df['review'][40000:], tokenizer)

df['sentiment'].replace('positive',1.,inplace=True)
df['sentiment'].replace('negative',0.,inplace=True)
one_hot_encoded = to_categorical(df['sentiment'].values)
y_train = one_hot_encoded[:40000]
y_test =  one_hot_encoded[40000:]

100%|██████████| 10000/10000 [00:38<00:00, 258.95it/s]


In [0]:
# Train the model
BATCH_SIZE = 8
EPOCHS = 1

# Use Adam optimizer to minimize the categorical_crossentropy loss
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Fit data to the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1)

model.save('nlp_model.h5') 

5000/5000 [==============================] - 2803s 561ms/step - loss: 0.2111 - accuracy: 0.9155 - val_loss: 0.1589 - val_accuracy: 0.9409


## Analysis of model performance

In [0]:
# Load the pretrained nlp_model
from tensorflow.keras.models import load_model
new_model = load_model('nlp_model.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [0]:
# Predict on test dataset
from sklearn.metrics import classification_report
pred_test = np.argmax(new_model.predict(X_test), axis=1)

In [0]:
print(classification_report(np.argmax(y_test,axis=1), pred_test))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      4960
           1       0.94      0.95      0.94      5040

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000



In [0]:
pred_test[:10]

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 1])

In [0]:
# Predicted 0 for the first review in the test dataset
df['review'][40000:40001]

13847    This was really a very bad movie. I am a huge fan of Italian Horror, Argento, Mario Bava, Fulci and yes, even our good friend here Lamberto sometimes comes out with a good one. I found the first two 'Demons' films to be highly entertaining - they were so bad they were great but this one is just so bad that it is really, really bad. It is intensely boring, the story never goes anywhere and I hated the characters - the wife slapping husband and whiny cry-baby pain in the *** wife drove me mad, there was nowhere near enough of the story devoted to the Ogre who was probably the best actor in the whole film. I turned it off about three quarters of the way through because I was very, very BORED! Don't bother.
Name: review, dtype: object

In [0]:
# Predicted 1 for the second review in the test dataset
df['review'][40001:40002]

16375    In a performance both volatile and graceful, Al Pacino re-teams with Sea of Love director, Harold Becker.<br /><br />As New York Mayor John Pappas in City Hall.<br /><br />A savvy thriller thats the first film ever shot inside the lower Manhattan structure that's ground zero for the City's government.<br /><br />That the other NYC locations provide the vivid settings as an idealistic mayoral aide (John Cusack) follows a trail of subversion and cover-up that may loop back to the man he serves and reveres.<br /><br />Bridget Fonda, Danny Aiello, Martin Landau, Tony Franciosa and David Paymer add more starry brilliance to this gripping tale of power.<br /><br />And the power behind power.
Name: review, dtype: object

In [0]:
# Predicted 1 for the third review in the test dataset
df['review'][40002:40003]

41263    Despite its budget limitations, this is a great film, proof that effort and imagination can overcome lack of cash. The opening, in which cave-paintings seem to show how some dinosaurs at least survived into the age of human beings, is a nice red herring. After that, a meteor comes down into a lake and causes heat which, in turn, causes the hatching of a frozen dinosaur egg (maybe the cave-paintings suggest instead that this isn't the first time such a thing has happened). When the prehistoric beast appears, it's a well-animated Plesiosaur which is soon causing disappearances in the local area. Alright, so it's not Jurassic Park, but it's still genuine entertainment for fans of monster movies.
Name: review, dtype: object